# **III. Proteins**
## The goal here is to scan the (filtered) MSA with the depolymerase DB 

## 1. hmmcan the MSA
## 2. Scan the results
***

> <b>1. hmmscan command

In [ ]:
import os
import random
import subprocess
from multiprocessing import Pool
import time
import pandas as pd
# Generate a list of path of filtered MSA
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_files = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

all_proteins = pd.read_csv(f"{path_files}/part_III_ptB/all_prophage_proteins.names.db.fasta", names = ["proteins"])

paths = [f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m" for prot in all_proteins["proteins"] 
         if os.path.isfile(f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m")==True 
         if os.path.isfile(f"{path_decipher}/{prot.split('__')[0]}/DBsuite_depo/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.suite.hhr")==False]


def scan_depolymerase(path_query) :
    path_db = "/home/conchae/databases/depolymerase_building/DBsuite_depolymerase/depolymerase_db.suite"
    phage = path_query.split("__")[1]
    query = path_query.split("/")[-1].split(".MSA")[0]
    path_dir_short = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo"
    path_dir = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo/{phage}"
    try :
        os.mkdir(path_dir_short)
    except FileExistsError :
        pass
    try :
        os.mkdir(path_dir)
    except FileExistsError :
        pass
    path_out = f"{path_dir}/{query}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d  {path_db} -o {path_out}.suite.hhr -blasttab {path_out}.suite.tab"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()
    with open(f"{path_files}/hhsearch_done","a+") as outfile :
        outfile.write(f"{path_query}\n")

if __name__ == '__main__':
    with Pool(30) as p:
        p.map(scan_depolymerase, paths)
        
        
# *********************************************************** 
#!/bin/bash
#BATCH --job-name=scan_ppt__
#SBATCH --qos=long
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=60 
#SBATCH --mem=200gb 
#SBATCH --time=10-00:00:00 
#SBATCH --output=scan_ppt__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/script_files/part_III/hhsearch.py

***
><b> 2. Scanning the outputs

In [ ]:
import os
#import random
#import subprocess
#from tqdm import tqdm
import time
import pandas as pd
#import pprint
import json 
#pp = pprint.PrettyPrinter(width = 150)
# Generate a list of path of filtered MSA 
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

out_names = ["query","target","x1","hmm","mismatch","gapopen","query_start","query_stop","target_start","target_stop","evalue","score"]

paths = [f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m" for prot in all_proteins["proteins"][0:20] 
         if os.path.isfile(f"{path_decipher}/{prot.split('__')[0]}/mmseqs_out/{prot.split('__')[1]}/{prot.split('.fasta')[0]}.MSA.a2m")==True]


def scan_depolymerase(path_query) :
    path_db = "/home/conchae/databases/depolymerase_building/DBsuite_Dpo.v3/DBsuite_Dpo.v3"
    phage = path_query.split("__")[1]
    query = path_query.split("/")[-1].split(".MSA")[0]
    path_dir_short = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo_v3"
    path_dir = f"{'/'.join(path_query.split('/')[0:-3])}/DBsuite_depo_v3/{phage}"
    try :
        os.mkdir(path_dir_short)
    except FileExistsError :
        pass
    try :
        os.mkdir(path_dir)
    except FileExistsError :
        pass
    path_out = f"{path_dir}/{query}"
    hhmscan_cmmd = f"hhsearch -i {path_query} -d  {path_db} -o {path_out}.suite.hhr -blasttab {path_out}.suite.tab"
    hhmscan_process = subprocess.Popen(hhmscan_cmmd, shell =True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    scan_out, scan_err = hhmscan_process.communicate()
    print(path_query , scan_out , scan_err , sep = "\n")
    with open(f"{path_files}/hhsearch_done_v3","a+") as outfile :
        outfile.write(f"{path_query}\n")


scores_out , dico_count_out, dico_ipr_out, depolymerase_dico_out = scan_tab(path_tabs["Paths"])

with open(f"{path_db}/dico_ipr_out.v3.json", "w") as outfile:
    json.dump(dico_ipr_out, outfile)
with open(f"{path_db}/dico_ipr_count.v3.json", "w") as outfile:
    json.dump(dico_count_out, outfile)    
with open(f"{path_db}/dbsuite_results.v3.json", "w") as outfile:
    json.dump(depolymerase_dico_out, outfile) 
    
scores_list = pd.DataFrame(scores_out)
scores_list.to_csv(f"{path_db}/scores.dbsuite.v3.txt", header=None, index=False)


***
# Build a DF with the dataframe

In [ ]:
df 1 : [protein name, sequence_index, KL type ancestor, ancestor_id, IPR, score, range alignment, sequence_aa]

df 2 : [sequence_index, sequence]

In [ ]:
import os 
import pandas as pd 
import json

path_labels = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"

f_labels=["Prophage_name","KL_ancestor","Id_monophyletic_group","Number of clades","Number of leafs","Number of new ancestors","Number of k-type swap","Nodes k-types","Nodes k-types all"]
df_prophages = pd.read_csv(f"{path_labels}/prophage_data.clusters_80.phageboost_70.tsv", sep="\t", names =f_labels) 

results_DBsuite = json.load(open(f"{path_db}/dbsuite_results.v3.json"))

def f_path_tab(query) :
    "From the protein name, get the tab output with score >= 20"
    path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
    strain , prophage = query.split("__")[0], query.split("__")[1]
    path_tab = f"{path_decipher}/{strain}/DBsuite_depo_v3/{prophage}/{query}.suite.tab"
    out_names = ["query","target","x1","hmm","mismatch","gapopen","consensus_start","consensus_stop","target_start","target_stop","evalue","score"]
    df_tab = pd.read_csv(f"{path_tab}", sep="\t", names= out_names)
    df_tab = df_tab[df_tab["score"] >= 20]
    sequence = open(f"{path_decipher}/{strain}/tmp/{prophage}/{query}.fasta").read().split("\n")[1]
    return df_tab, sequence

def get_qlimits(query,target) :
    strain , prophage = query.split("__")[0], query.split("__")[1]
    df_tab, sequence = f_path_tab(query)
    #print(df_tab, target)
    #return (1,2)
    consensus_start = df_tab[df_tab["target"]==f"{target}"]["consensus_start"].values[0] 
    consensus_stop = df_tab[df_tab["target"]==f"{target}"]["consensus_stop"].values[0]
    msa_seq = "".join(open(f"{path_decipher}/{strain}/mmseqs_out/{prophage}/{query}.MSA.a2m").read().split(">")[1].split("\n")[1:])
    i = 0
    for n in range(len(msa_seq)) :
        if msa_seq[n] == "-" :
            pass
        else :
            i += 1 
        if n == int(consensus_start)-1 :
            qstart = i
        elif n == int(consensus_stop)-1 :
            qstop = i 
            break            
    return qstart, qstop
        
# df 1 : [protein name, #sequence_index# , KL type ancestor, ancestor_id, IPR, score, qstart, qstop, sequence_aa]
with open(f"{path_db}/Results_III_DataFrame.v3.csv", "w") as outfile :
    for protein in tqdm(results_DBsuite) :
        df_tab , sequence = f_path_tab(protein)
        prophage = "__".join(protein.split("__")[0:2])+".fasta"
        df_prot = df_prophages[df_prophages["Prophage_name"]==prophage]
        kl_ancestor = df_prot["KL_ancestor"].values[0]
        ancestor_id = df_prot["Id_monophyletic_group"].values[0]
        #print(prophage, kl_ancestor , ancestor_id)
        for index_tab, hit_ipr in df_tab.iterrows() :
            target = hit_ipr["target"]
            start , stop = get_qlimits(protein,target)
            score = hit_ipr["score"]
            if stop - start >= 30 :
                outfile.write(f"{protein}\t{kl_ancestor}\t{ancestor_id}\t{target}\t{score}\t{start}\t{stop}\t{sequence}\n")
                #print(f"{protein}\t{kl_ancestor}\t{ancestor_id}\t{target}\t{score}\t{start}\t{stop}\n")         
                
                
labels_results = ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence"]
results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.csv", sep="\t", names= labels_results)

sequences_df = results_df["sequence"].unique()
with open(f"{path_db}/Results_III_sequences.v3.csv", "w") as outfile :
    for index, sequence in enumerate(sequences_df) :
        print(sequence)
        outfile.write(f"{index}\t{sequence}\n")

# *******************
# DF with index sequences :

import pandas as pd 
import os 
path_labels = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"

labels_results = ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence"]
results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.csv", sep="\t", names= labels_results)
sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.v3.csv", sep = "\t" ,names= ["index", "sequence"])

results_df["index_seq"] = sequence_df[sequence_df["sequence"]== results_df["sequence"]]["index"]

index_list = []
for index , row in tqdm(results_df.iterrows()) :
    index_seq = sequence_df[sequence_df["sequence"] == row["sequence"]]["index"].values[0]
    index_list.append(index_seq)
    
results_df["index_seq"] = index_list
results_df.to_csv(f"{path_db}/Results_III_DataFrame.v3.final.csv", sep="\t",header =  ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence","index_seq"], index = False )



In [ ]:
#!/bin/bash
#BATCH --job-name=writting_r__
#SBATCH --qos=short
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=5 
#SBATCH --mem=20gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=writting_r__%j.log 

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/script_files/part_III/writting_db3.py